**Import Required Libraries**

In [ ]:
import sys,os
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten  
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy  
from keras.optimizers import Adam  
from keras.regularizers import l2 
from keras.utils import np_utils

**Loading, Reading, and Pre-Processing Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/My Drive/Emotion Detection")

In [ ]:
os.listdir()

['archive.zip', 'm.json', 'Detection_Emotion.h5', 'Emotion_Detection_1.ipynb']

In [ ]:
df = pd.read_csv("archive.zip")
print(df)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [ ]:
X_train,train_y,X_test,test_y=[],[],[],[]  

for index, row in df.iterrows():  
    val=row['pixels'].split(" ")  
    if 'Training' in row['Usage']:
      X_train.append(np.array(val,'float32'))  
      train_y.append(row['emotion'])  
    elif 'PublicTest' in row['Usage']:  
      X_test.append(np.array(val,'float32'))  
      test_y.append(row['emotion']) 

In [ ]:
num_features = 64  
num_labels = 7  
batch_size = 64  
epochs = 175
width, height = 48, 48  


X_train = np.array(X_train,'float32')  
train_y = np.array(train_y,'float32')  
X_test = np.array(X_test,'float32')  
test_y = np.array(test_y,'float32')  

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)  
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)  

In [ ]:
X_train -= np.mean(X_train, axis=0)  
X_train /= np.std(X_train, axis=0)  

X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0)  

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)  

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1) 

**Building Model**

In [ ]:
model = Sequential()  

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:]), padding='same'))  
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu', padding='same'))  
model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2)))  
model.add(Dropout(0.3))  

#2nd convolution layer  
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))  
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))  
model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2)))  
model.add(Dropout(0.3))  

#3rd convolution layer  
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))  
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))  
model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2)))  

model.add(Flatten())  

#fully connected neural networks  
#model.add(Dense(1024, activation='relu'))  
#model.add(Dropout(0.2))  
#model.add(Dense(1024, activation='relu'))  
#model.add(Dropout(0.2))  

model.add(Dense(num_labels, activation='softmax'))  

model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        3

**Compiling and Training Model**

In [ ]:
model.compile(loss=categorical_crossentropy,  
              optimizer=Adam(),  
              metrics=['accuracy'])  

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy')>=1.00):
                print("Reached 100% accuracy so cancelling training!")
                self.model.stop_training = True

callback = myCallback()

In [ ]:
model.fit(X_train, train_y, 
          steps_per_epoch=len(X_train) / batch_size, 
          batch_size=batch_size,  
          epochs=epochs,  
          verbose=1,  
           callbacks=[callback],
          validation_data=(X_test, test_y)
          )  

Epoch 1/175
449/448 [==============================] - 21s 47ms/step - loss: 1.8567 - accuracy: 0.3392 - val_loss: 1.7904 - val_accuracy: 0.3413
Epoch 2/175
449/448 [==============================] - 20s 46ms/step - loss: 1.5181 - accuracy: 0.4424 - val_loss: 1.4303 - val_accuracy: 0.4921
Epoch 3/175
449/448 [==============================] - 20s 45ms/step - loss: 1.3464 - accuracy: 0.5004 - val_loss: 1.3037 - val_accuracy: 0.5127
Epoch 4/175
449/448 [==============================] - 21s 46ms/step - loss: 1.2346 - accuracy: 0.5389 - val_loss: 1.3261 - val_accuracy: 0.5088
Epoch 5/175
449/448 [==============================] - 20s 46ms/step - loss: 1.1319 - accuracy: 0.5797 - val_loss: 1.2096 - val_accuracy: 0.5570
Epoch 6/175
449/448 [==============================] - 21s 46ms/step - loss: 1.0737 - accuracy: 0.6008 - val_loss: 1.1866 - val_accuracy: 0.5770
Epoch 7/175
449/448 [==============================] - 21s 46ms/step - loss: 0.9985 - accuracy: 0.6278 - val_loss: 1.1647 - val_ac

**Save Model**

In [ ]:
m_json = model.to_json()  
with open("m.json", "w") as json_file:  
    json_file.write(m_json)  
model.save_weights("Detection_Emotion.h5") 

## After Saving the model, I’ll use Opencv to predict Emotions in Real-Time. But first of all, download the saved weights and model. We’ll work on our local machines now. why?? Because VideoCapture doesn’t work in google colab, in other words, using a webcam in google colab isn’t easy.


Go to Real_Time_Detection.py file.